In [1]:
# Boiler plate stuff to start the module
import jpype
import jpype.imports
from jpype.types import *
import sys 
import numpy as np
import traceback
import random
import Client
import copy

C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\AStA\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Launch the JVM
#jpype.shutdownJVM()
#try:
#    jpype.startJVM(classpath=['./maze-server-v2019.4.jar'])
#except:
#    print("jvm already running")

In [3]:
# import the Java modules
from java.io import IOException
from java.net import Socket
from java.net import UnknownHostException

from javax.net.ssl import SSLSocket
from javax.net.ssl import SSLSocketFactory
from javax.xml.bind import JAXBException
from javax.xml.bind import UnmarshalException

from de.fhac.mazenet.server.generated import AwaitMoveMessageData
from de.fhac.mazenet.server.generated import Errortype
from de.fhac.mazenet.server.generated import MazeCom
from de.fhac.mazenet.server.generated import MazeComMessagetype
from de.fhac.mazenet.server.generated import MoveMessageData
from de.fhac.mazenet.server.generated import BoardData

from de.fhac.mazenet.server.networking import MazeComMessageFactory
from de.fhac.mazenet.server.networking import XmlInputStream
from de.fhac.mazenet.server.networking import XmlOutputStream
from de.fhac.mazenet.server.game import Board
from de.fhac.mazenet.server.game import Position
from de.fhac.mazenet.server.game import Card

In [4]:
HOST = "127.0.0.1"
PORT = 9888

In [5]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
from mss import mss
import pydirectinput
import cv2
import pytesseract
import matplotlib.pyplot as plt
import time

In [7]:
ls_range = np.arange(1,6,2)
potentialShiftMoves =  []
potentialShiftMovesPos =  []
for i in ls_range:
    potentialShiftMoves.append((0, i))
    potentialShiftMoves.append((6, i))
    potentialShiftMoves.append((i, 0))
    potentialShiftMoves.append((i, 6))
    potentialShiftMovesPos.append(Position(0, i))
    potentialShiftMovesPos.append(Position(6, i))
    potentialShiftMovesPos.append(Position(i, 0))
    potentialShiftMovesPos.append(Position(i, 6))

In [8]:
class MazeEnv(Env):
    def __init__(self):
        super().__init__()
        # Actions we can take, rotate card left, rotate card right, push card, go up, down, left, right, no op
        # actions: push card, move, no op
        # self.action_space = Discrete(3)
        # actions: pick x and y where the next card should be pushed and rotation
        # self.action_space = Box(low=np.array([1,1,0]), high=np.array([5,5,3]), dtype=np.int8)
        # actions: pick a shiftposition from the lot, the rotation of the card, the x_move and y_move
        # self.action_space = Box(low=np.array([0,0]), high=np.array([11,3]), dtype=np.int8)
        self.action_space = gym.spaces.MultiDiscrete([12, 4, 7, 7], dtype=np.int8)
        #  array
        #self.observation_space = Tuple((
        #                             Discrete(50),
        #                             Discrete(24),
        #                             Discrete(2))
        #                            )
        #self.observation_space = Box(low=0, high=255, shape=(1,800,1100), dtype=np.uint8)
        #self.cap = mss()
        #self.game_location = {'top':200, 'left':100, 'width':1700, 'height':800}
        #self.end_location = {'top':50, 'left':400, 'width':400, 'height':150}
        # player position, next treasure position
        self.observation_space = Tuple((Box(low=0, high=6, shape=(1,2), dtype=np.uint8),
                                       Box(low=0, high=6, shape=(1,2), dtype=np.uint8)
                                       )) 
        self.client = None
        self.player_pos = Box(low=0, high=6, shape=(1,2))
        self.next_treasure_pos = Box(low=0, high=6, shape=(1,2))
        self.is_done = False

    def moeglicheOrientierungen(self, rot, card):
        card_ = Card(card)
        openings = card_.getOpenings()
        if(card.getShape() == Card.CardShape.I):
            openings.setBottom(not openings.isBottom())
            openings.setLeft(not openings.isLeft())
            openings.setRight(not openings.isRight())
            openings.setTop(not openings.isTop())
            return  Card(card_)
        else:
            if(rot == 0):
                return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
            if(rot == 1):
                return Card(card.getShape(), Card.Orientation.D90, card.getTreasure())
            if(rot == 2):
                return Card(card.getShape(), Card.Orientation.D180, card.getTreasure())
            if(rot == 3):
                return Card(card.getShape(), Card.Orientation.D270, card.getTreasure())
        return Card(card.getShape(), Card.Orientation.D0, card.getTreasure())
    
    def Move(self, awaitMove, action):
        reward = 0
        boardData = awaitMove.getBoard()
        treasure = awaitMove.getTreasureToFindNext()
        board = Board(boardData)
        playerPosition = board.findPlayer(self.client.getId())
        psm = potentialShiftMoves[action[0]]
        position = Position(psm[0], psm[1])
        
        potentialMove = MoveMessageData()
        psmp = copy.copy(potentialShiftMovesPos)
        print("len psmp:", len(psmp))
        print("full psmp:", psmp)
        #potentialShiftMoves.append(position)
        print("forbidden:", board.getForbidden())
        if(position == board.getForbidden()):
            print("forbidden shift position, random shift position used, reward decreased")
            print("forbid pos#:", position)
            reward -= 10
            
            try:
                psmp.remove(position)
            except Exception:
                print("element not in list, skipping")
            print("psmp[0]", psmp[0])
            potentialMove.setShiftPosition(psmp[0])
        else:
            potentialMove.setShiftPosition(position)
        
        treasurePositionData = board.findTreasure(treasure)
        
        orientedShiftCard = self.moeglicheOrientierungen(action[1], board.getShiftCard())
        
        potentialMove.setShiftCard(orientedShiftCard)
        potentialMove.setNewPinPos(playerPosition)

        #if len(potentialShiftMoves) == 0:
        #    print("forbidden move, reward decreased")
        #    self.next_treasure_pos = treasurePositionData
        #    self.player_pos = playerPosition
        #    return potentialMove, -20

        boardNext = board.fakeShift(potentialMove)
        treasurePositionData = boardNext.findTreasure(treasure)
        
        new_player_pos = boardNext.findPlayer(self.client.getId())
        print("find player:", new_player_pos)
        reachablePositions = boardNext.getAllReachablePositions(new_player_pos)
        rp = np.array(reachablePositions)
        print("the number of reachablePositions:", rp.shape)
        #chosen_position = all_moves[action[2]]
        chosen_pos_obj = Position(action[2], action[3])
        chosen_pos_arr = np.array(action[2], action[3])
        print("chosenposoibj:", chosen_pos_obj)
        if(treasurePositionData == None):
            print("no treasure? skipping")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos = eval(treasurePositionData.toString())
            self.player_pos = eval(new_player_pos.toString())
            return potentialMove, reward

        #if(chosen_pos_obj == board.getForbidden()):
        #    print("forbidden move, reward significantly decreased!")
        #    potentialMove.setNewPinPos(new_player_pos)
        #    self.next_treasure_pos = treasurePositionData
        #    self.player_pos = new_player_pos
        #    return potentialMove, -20

        if(chosen_pos_obj not in rp):
            print("cannot reach position, reward decreased")
            potentialMove.setNewPinPos(new_player_pos)
            self.next_treasure_pos = eval(treasurePositionData.toString())
            self.player_pos = eval(new_player_pos.toString())
            return potentialMove, reward - 10
        else:
            print("treasure position data:", treasurePositionData)
            treasurePosition = Position(treasurePositionData)
            print("treasure pos:", treasurePosition)

            if(chosen_pos_obj == treasurePosition):
                print("straight to the treasure, reward increased!")
                potentialMove.setNewPinPos(treasurePosition)
                self.next_treasure_pos = eval(treasurePositionData.toString())
                self.player_pos = eval(treasurePosition.toString())
                return potentialMove, reward + 20
            else:
                print("reachable position, but no treasure")
                potentialMove.setNewPinPos(chosen_pos_obj)
                self.next_treasure_pos = eval(treasurePositionData.toString())
                self.player_pos = eval(chosen_pos_obj.toString())
                return potentialMove, reward - 5

        print("Object not reachable: random move :/")
        potentialMove.setNewPinPos(reachablePositions[0])
        self.next_treasure_pos = eval(treasurePositionData.toString())
        self.player_pos = eval(reachablePositions[0].toString())
        return potentialMove, reward - 10
    
    def awaitMove(self, receivedMazeCom, action):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        move, reward = self.Move(awaitMove, action)
        mazeComToSend = MazeCom()
        mazeComToSend.setId(self.client.getId())
        mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
        mazeComToSend.setMoveMessage(move)
        self.client.out.write(mazeComToSend)
        return reward
    
    def step(self, action):
        reward = 0
        print("ruun")
        try:
            receivedMazeCom = self.client.in_.readMazeCom()
            if(receivedMazeCom.getMessagetype() == MazeComMessagetype.LOGINREPLY):
                newid = receivedMazeCom.getLoginReplyMessage().getNewID()
                self.client.setId(newid)
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                print(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                reward += 2
            elif (receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                print(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
                reward -= 10
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                reward += self.awaitMove(receivedMazeCom, action)
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                print("in MoveInfo")
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                print("You have won")
                self.is_done = True
                reward += 100
            else:
                print("Unknown message type: " + receivedMazeCom.getMessagetype())
        except Exception as e:
            print(traceback.format_exc())
            if str(type(e)) == "<java class 'java.net.SocketException'>":
                print("socket error, disconnected")
                sys.exit(1)
            elif str(type(e)) == "<java class 'java.io.EOFException'>":
                print("end reached, disconnecting")
                sys.exit(1)
            print(e)
        #############
        done = self.get_done() 
        observation = self.get_observation()
        info = {}
        return observation, reward, done, info
        
    def render(self):
        pass
    
    def reset(self):
        print("reeeset time :]")
        self.is_done = False
        #while True:
        try:
            login = MazeComMessageFactory.createLoginMessage(self.client.name)
            self.client.out.write(login)
            #break
        except:
            print("failed login :)")
            #self.client = Client.Client(self.client.name, self.client.address, self.client.port, self.client.tls)
        return self.get_observation()
    
    # Closes the game?
    def close(self):
        pass
    
    def get_observation(self):
        return (self.player_pos, self.next_treasure_pos)
    
    def get_done(self):
        return self.is_done

In [9]:
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy

In [10]:
menv = MazeEnv()

In [11]:
menv.client = Client.Client('joe', HOST, PORT, False)

In [12]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

import supersuit as ss
from ray.tune.registry import register_env
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray import tune

In [13]:
def env_creator(args):
    env = MazeEnv()
    env.client = Client.Client('joe', HOST, PORT, False)
    return env

In [14]:
env_name = "rl_env_para"

register_env(env_name, lambda config: env_creator(config))

test_env = env_creator({})
obs_space = test_env.observation_space
act_space = test_env.action_space

In [15]:
def gen_policy(i):
    config = {
        "gamma": 0.99,
    }
    return (None, obs_space, act_space, config)

In [16]:
policies = {"policy_0": gen_policy(0)}
policy_ids = list(policies.keys())

In [ ]:
tune.run(
    "PPO",
    name="PPO",
    stop={"timesteps_total": 1000},
    checkpoint_freq=10,
    local_dir="./ray_results/"+env_name,
    config={# Environment specific
    "env": env_name,
    # https://github.com/ray-project/ray/issues/10761
    #"no_done_at_end": True,
    # "soft_horizon" : True,
    #"num_gpus": 0,
    #"num_workers": 1,
    #"num_envs_per_worker": 1,
    #"compress_observations": False,
    #"batch_mode": 'truncate_episodes',
    #"clip_rewards": False,
    #"vf_clip_param": 500.0,
    #"entropy_coeff": 0.01,
    # effective batch_size: train_batch_size * num_agents_in_each_environment [5, 10]
    # see https://github.com/ray-project/ray/issues/4628
    #"train_batch_size": 1000,  # 5000
    #"rollout_fragment_length": 50,  # 100
    #"sgd_minibatch_size": 100,  # 500
    #"vf_share_layers": False
        },
)

2022-11-15 16:40:26,744	INFO worker.py:1528 -- Started a local Ray instance.


In [ ]:
path = 'Training/Logs'
model = A2C('MlpPolicy', menv, verbose=1, tensorboard_log=path)

In [ ]:
model.learn(total_timesteps=100)

In [ ]:
a2c_path = "Training/Saved Models/A2C_100k_Mazenet_Model"
model.save(a2c_path)

In [ ]:
qa

In [ ]:
try:
    c1 = Client.Client("ay", HOST, PORT, False)
    c1.run()
except Exception:
    print(traceback.format_exc())

In [ ]:
menv = MazeEnv()

In [ ]:
menv.client = Client.Client('joe', HOST, PORT, False)

In [ ]:
obs = menv.reset()
done = False  
total_reward   = 0
while not done:
    print(menv.action_space.sample())
    obs, reward,  done, info =  menv.step(menv.action_space.sample())
    total_reward  += reward
print('Total Reward for episode {} is {}'.format(episode, total_reward))

In [ ]:
qa

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import env_checker

In [ ]:
menv = MazeEnv()

In [ ]:
menv.client = Client('joe', HOST, PORT, False)

In [ ]:
#env_checker.check_env(menv)

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

In [ ]:

c1 = Client('joe2', HOST, PORT, False)
c1.login()

c = Client('joe', HOST, PORT, False)
c.login()

c_list = [c, c1]

for agent in c_list:
    agent.play()
    time.sleep(1)


In [ ]:
d

In [ ]:
x

In [ ]:
xxc